In [ ]:
from openai import OpenAI
import requests
import time
import json

client = OpenAI(api_key= "YOUR-API-KEY")

use chatgpt to understand & sort questions into categories

In [3]:
def remove_json_tag_if_exists(s: str) -> str:
    if s.startswith("```json\n"):
        s = s.replace("```json\n", "", 1)
    elif s.startswith("```\n"):
        s = s.replace("```\n", "", 1)

    if s.endswith("```"):
        s = s[:-3]

    return s.strip()


In [4]:
def interpret_question(question):
    system_prompt = """
You are a bioinformatician. Your job is to classify the user's question and extract relevant data.
Return one of the following JSON formats:

// 0. Python code generation, do this only when the user explicitly asked for python code
{
  "is_code_task": true,
  "needs_alignment": false,
  "needs_sequence_fetch": false,
  "needs_snp_lookup": false,
  "program": null,
  "sequence": null,
  "answer": null,
  "code": "Python code string"
}

// 1. Sequence alignment (e.g., BLAST), when the user asked for sequence alignment
{
  "is_code_task": false,
  "needs_alignment": true,
  "needs_sequence_fetch": false,
  "needs_snp_lookup": false,
  "program": "blastn" or "blastp" or "blastx" etc.,
  "sequence": ">myseq\\nACTG...",
  "answer": null
}

// 2. Sequence fetch from genomic region (e.g., chr12:10000-20000).
If the user provides a genomic region (e.g., "chr12:10000-20000" or "12:10000-20000"), convert the chromosome label into the correct GRCh38 RefSeq accession and return the following JSON structure:
{
  "is_code_task": false,
  "needs_alignment": false,
  "needs_sequence_fetch": true,
  "needs_snp_lookup": false,
  "program": null,
  "sequence": "NC_000012.12:10000-20000",
  "answer": null
}


// 3. SNP lookup (e.g., rs123456)
{
  "is_code_task": false,
  "needs_alignment": false,
  "needs_sequence_fetch": false,
  "needs_snp_lookup": true,
  "program": null,
  "sequence": "rs123456",
  "answer": null
}

// 4. General question
{
  "is_code_task": false,
  "needs_alignment": false,
  "needs_sequence_fetch": false,
  "needs_snp_lookup": false,
  "program": null,
  "sequence": null,
  "answer": "Direct natural language answer"
}
"""


    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
        temperature=0
    )
    
    content = response.choices[0].message.content
    
    #print("Received content:", repr(content))  

    content = remove_json_tag_if_exists(content)
    
    #print("Cleaned content:", repr(content))  
    
    if not content.strip():
        print("Received empty or whitespace content")
        raise ValueError("Content is empty, cannot parse as JSON.")
    
    try:
        return json.loads(content)
    except json.JSONDecodeError as e:
        print("Failed to parse GPT response as JSON:")
        print(content)  # Print the actual content causing the errors
        raise e
    except Exception as e:
        print("An unexpected error occurred:", e)
        raise e

call gpt api again to summarize output

In [6]:
def summarize_blast_with_gpt(xml_result):
    prompt = """
You are a bioinformatics assistant. Below is a BLAST alignment result in XML format.
Summarize the top 1–3 alignments in plain, human-readable language. For each alignment, include:
-The matched gene and species (if available)
-The chromosome (in the format “chromosome X”, if provided)
-The gene's location on the chromosome (if available)
-Identity percentage, alignment length, and E-value

Use scientific language suitable for a molecular biology researcher.

XML input:
""" + xml_result[:15000]  # truncate long XMLs for safety

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You interpret NCBI BLAST results."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )

    return response.choices[0].message.content


In [7]:
def summarize_snp_with_gpt(snp_info: dict, rsid: str) -> str:
    prompt = f"""
You are a bioinformatics assistant. Below is information about a SNP.
Please show it clearly with a brief summarization of the result.

SNP Info:
- dbSNP ID: {rsid}
- Chromosome: {snp_info.get('chr', 'N/A')}
- Position: {snp_info.get('position', 'N/A')}
- Gene: {snp_info.get('gene', 'N/A')}
- Function: {snp_info.get('function', 'N/A')}
- Clinical Significance: {snp_info.get('clinical', 'N/A')}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You summarize SNP annotations."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )

    return response.choices[0].message.content


call ncbi api (blast/E utility) to complete tasks

In [ ]:
def run_blast(sequence, program="blastn", database="nt"):
    blast_url = "https://blast.ncbi.nlm.nih.gov/Blast.cgi"
    
    res = requests.post(blast_url, data={
        "CMD": "Put",
        "PROGRAM": program,
        "DATABASE": database,
        "QUERY": sequence
    })
    res.raise_for_status()

    rid, rtoe = None, 10
    for line in res.text.splitlines():
        if "RID =" in line:
            rid = line.split("=")[1].strip()
        elif "RTOE =" in line:
            rtoe = int(line.split("=")[1].strip())

    if not rid:
        raise Exception("BLAST job failed — no RID found.")

    print(f"RID: {rid} | Waiting ~{rtoe} seconds...")
    time.sleep(rtoe)

    while True:
        check = requests.get(blast_url, params={
            "CMD": "Get",
            "RID": rid,
            "FORMAT_OBJECT": "SearchInfo"
        })

        if "Status=READY" in check.text:
            break
        elif "Status=FAILED" in check.text:
            raise Exception("BLAST job failed.")
        print("Waiting for BLAST to finish...")
        time.sleep(5)

    result = requests.get(blast_url, params={
        "CMD": "Get",
        "RID": rid,
        "FORMAT_TYPE": "XML"
    })

    print("BLAST result retrieved.")
    return result.text


In [9]:
def fetch_snp_info(rsid):
    """
    Looks up SNP info via NCBI E-utilities esummary.fcgi.
    """
    rsid_clean = rsid.lower().replace("rs", "")
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
    params = {
        "db": "snp",
        "id": rsid_clean,
        "retmode": "json"
    }
    res = requests.get(url, params=params)
    res.raise_for_status()
    data = res.json()

    docsum = data["result"].get(rsid_clean)
    if not docsum:
        raise Exception(f"No info found for SNP {rsid}")
    
    genes = docsum.get("genes", [])
    if genes:
        gene_name = genes[0].get("name", "Unknown")
        gene_id = genes[0].get("gene_id", "Unknown")
    else:
        gene_name = "None listed"
        gene_id = "N/A"

    return {
        "rsid": rsid,
        "chr": docsum.get("chr"),
        "position": docsum.get("chrpos"),
        "alleles": docsum.get("allele"),
        "gene": gene_name,
        "gene_id": gene_id,
        "snp_class": docsum.get("snp_class")
    }


In [10]:
def fetch_genomic_sequence(region, strand=1):
    """
    Fetch DNA sequence from a specified region using GRCh38 coordinates via NCBI E-utilities efetch.fcgi.
    Region format: "NC_000001.11:91950805-91950932"
    """
    refseq_id, coords = region.split(":")
    start, end = map(int, coords.split("-"))

    fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    fetch_params = {
        "db": "nuccore",
        "id": refseq_id,
        "rettype": "fasta",
        "retmode": "text",
        "seq_start": start,
        "seq_stop": end,
        "strand": strand
    }
    response = requests.get(fetch_url, params=fetch_params)
    response.raise_for_status()

    return response.text


In [11]:
def process_question(question):
    parsed = interpret_question(question)

    if parsed.get("is_code_task", False):
        return f"```python\n{parsed['code']}\n```"

    elif parsed.get("needs_alignment", False):
        sequence = parsed["sequence"]
        program = parsed["program"]
        result_xml = run_blast(sequence, program=program)
        return summarize_blast_with_gpt(result_xml)
    
    elif parsed.get("needs_sequence_fetch", False):
        region = parsed["sequence"]
        try:
            sequence = fetch_genomic_sequence(region)
            return f"**The DNA sequence is:**\n\n{sequence.strip()}"
        except Exception as e:
            return f"Failed to fetch sequence for `{region}`: {e}"


    elif parsed.get("needs_snp_lookup", False):
        rsid = parsed["sequence"]
        try:
            snp_info = fetch_snp_info(rsid)
            return summarize_snp_with_gpt(snp_info, rsid)
        except Exception as e:
            return f"Failed to fetch SNP info for {rsid}: {str(e)}"

    else:
        return parsed.get("answer", "No answer generated.")


### Example - CSV usage

In [ ]:
import pandas as pd
from tqdm import tqdm
import time

df = pd.read_csv("GPTs(2)/snp_gene.csv")  

for col in ["Answer1", "Answer2", "Answer3"]:
    if col not in df.columns:
        df[col] = ""

for i, row in tqdm(df.iterrows(), total=len(df)):
    question = row["Question"]

    for j in range(1, 4):
        try:
            answer = process_question(question)
            df.at[i, f"answer{j}"] = answer
            time.sleep(1)  # slight delay for safety
        except Exception as e:
            df.at[i, f"answer{j}"] = f"[Error]: {str(e)}"
            print(f"Error on answer{j} for row {i}: {e}")

df.to_csv("GPTs(2)/snp_gene_FINISH.csv", index=False)
print("All done!!")


### Example - Single Usage

In [12]:
question_snp = "Which gene is SNP rs1217074595 associated with?"

process_question(question_snp)

'**SNP Summary**\n\n- **dbSNP ID:** rs1217074595\n- **Chromosome Location:** Chromosome 20\n- **Genomic Position:** 20:50298395\n- **Associated Gene:** LINC01270\n- **Function:** Not Available\n- **Clinical Significance:** Not Available\n\n**Brief Summary:**\nThe SNP rs1217074595 is located on chromosome 20 at position 50298395 and is associated with the gene LINC01270. Currently, there is no available information regarding its specific function or clinical significance.'

In [13]:
question_programming = "Only generate Python programming code, do not provide any explanation to answer which organism does the DNA sequence come from:AGGGGCAGCAAACACCGGGACACACCCATTCGTGCACTAATCAGAAACTTTTTTTTCTCAAATAATTCAAACAATCAAAATTGGTTTTTTCGAGCAAGGTGGGAAATTTTTCGAT"
process_question(question_programming)

'```python\n# Python code to identify organism from DNA sequence\nfrom Bio import Entrez, SeqIO\n\n# Set email for Entrez\nEntrez.email = "your.email@example.com"\n\n# Function to search for the organism\n\ndef search_organism(sequence):\n    handle = Entrez.esearch(db="nucleotide", term=sequence, retmax=1)\n    record = Entrez.read(handle)\n    handle.close()\n    if record[\'IdList\']:\n        seq_id = record[\'IdList\'][0]\n        handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="gb", retmode="text")\n        seq_record = SeqIO.read(handle, "genbank")\n        handle.close()\n        return seq_record.annotations[\'organism\']\n    else:\n        return "Organism not found"\n\n# Example usage\nsequence = "AGGGGCAGCAAACACCGGGACACACCCATTCGTGCACTAATCAGAAACTTTTTTTTCTCAAATAATTCAAACAATCAAAATTGGTTTTTTCGAGCAAGGTGGGAAATTTTTCGAT"\norganism = search_organism(sequence)\nprint(f"The DNA sequence is from: {organism}")\n```'

In [14]:
question_ontology = "What is the gene ontology term associated with SLC27A5, AKT1, AKT2, ACSL1, SLC27A1, ACSL5, SLC2A1, THBS1, IRS2, CD36?"
process_question(question_ontology)

'Gene Ontology (GO) terms are used to describe the functions, processes, and cellular components associated with genes. Each of the genes listed (SLC27A5, AKT1, AKT2, ACSL1, SLC27A1, ACSL5, SLC2A1, THBS1, IRS2, CD36) may be associated with multiple GO terms. To find the specific GO terms for each gene, you can use resources like the Gene Ontology Consortium website or databases such as UniProt or NCBI Gene. These resources provide detailed annotations for each gene, including their associated GO terms.'

In [16]:
question_amino = "Nucleic acid sequence of a human gene: atggttctaaataaaaaatatgggagctaccttggtgtcaacttgggttttggcttcggagtcaccatgggagtgcacgtggcaggccgcatctctggagcccacatgaacgcagctgtgacctttgctaactgtgcgctgggccgcgtgccctggaggaagtttccggtctatgtgctggggcagttcctgggctccttcctggcggctgccaccatctacagtctcttctacacggccattctccacttttcgggtggacagctgatggtgaccggtcccgtcgctacagctggcatttttgccacctaccttcctgatcacatgacattgtggcggggcttcctgaatgaggcgtggctgaccgggatgctccagctgtgtctcttcgccatcacggaccaggagaacaacccagcactgccaggaacagaggcgctggtgataggcatcctcgtggtcatcatcggggtgtcccttggcatgaacacaggatatgccatcaacccgtcccgggacctgcccccccgcatcttcaccttcattgctggttggggcaaacaggtcttcaggtggcatcatctacctggtcttcattggctccaccatcccacgggagcccctgaaattggaggattctgtggcgta. Translate the above nucleic acid sequences to the corresponding amino acid sequences."

process_question(question_amino)

'The translation of the given nucleic acid sequence into an amino acid sequence can be done using a codon table. Each set of three nucleotides (codon) corresponds to a specific amino acid. Here is the translation of the provided sequence:\n\nMVSNKKIWELPLVSTWVFGLGVTMGVHVAGRISEPTCTQLYFANCAAGRVPGGVSGSYVLGQFLGSSWRAATISTLSTPPISTFSGWTADGDRSRRYSGIFATYLFITDTVAGAFLNEAVADRDAPAVSFAYTDQENNPALQETEGAGDRHPVVIIAGVPLAMNTGYAINTVPGTCPPASSTFIAWLGQTVFSGGIIYLVFIASTIPREPLKLEDFVAV'

In [15]:
question = "Align the DNA sequence to the human genome:ATTCTGCCTTTAGTAATTTGATGACAGAGACTTCTTGGGAACCACAGCCAGGGAGCCACCCTTTACTCCACCAACAGGTGGCTTATATCCAATCTGAGAAAGAAAGAAAAAAAAAAAAGTATTTCTCT"
process_question(question)

RID: 7VEBSX7B015 | Waiting ~30 seconds...
BLAST result retrieved.


'The BLAST alignment results reveal the following top hits:\n\n1. **Homo sapiens SLCO3A1 Gene**\n   - **Species**: Homo sapiens\n   - **Gene**: Solute carrier organic anion transporter family member 3A1 (SLCO3A1)\n   - **Chromosome**: Not explicitly mentioned\n   - **Gene Location**: Positions 100090 to 100217 on the sequence\n   - **Identity Percentage**: 100%\n   - **Alignment Length**: 128 base pairs\n   - **E-value**: 1.07566e-56\n\n2. **Eukaryotic Synthetic Construct**\n   - **Species**: Synthetic construct\n   - **Chromosome**: Chromosome 15\n   - **Gene Location**: Positions 72494035 to 72494162 on the sequence\n   - **Identity Percentage**: 100%\n   - **Alignment Length**: 128 base pairs\n   - **E-value**: 1.07566e-56\n\n3. **Homo sapiens Chromosome 15 Clone RP11-24J19**\n   - **Species**: Homo sapiens\n   - **Chromosome**: Chromosome 15\n   - **Gene Location**: Positions 94509 to 94636 on the sequence\n   - **Identity Percentage**: 100%\n   - **Alignment Length**: 128 base pai

In [17]:
question_species = "Which organism does the DNA sequence come from:AGGGGCAGCAAACACCGGGACACACCCATTCGTGCACTAATCAGAAACTTTTTTTTCTCAAATAATTCAAACAATCAAAATTGGTTTTTTCGAGCAAGGTGGGAAATTTTTCGAT"


process_question(question_species)

RID: 7VEEGWXE015 | Waiting ~30 seconds...
BLAST result retrieved.


'Here is a summary of the top three alignments from the BLAST results:\n\n1. **Caenorhabditis elegans CGC1 DNA, chromosome IV**\n   - **Species:** Caenorhabditis elegans\n   - **Chromosome:** Chromosome IV\n   - **Location on Chromosome:** Positions 13,137,915 to 13,138,029\n   - **Identity Percentage:** 100%\n   - **Alignment Length:** 115 base pairs\n   - **E-value:** 1.052e-49\n\n2. **Caenorhabditis elegans Cosmid K09E10**\n   - **Species:** Caenorhabditis elegans\n   - **Chromosome:** Not specified\n   - **Location on Chromosome:** Positions 10,623 to 10,737\n   - **Identity Percentage:** 100%\n   - **Alignment Length:** 115 base pairs\n   - **E-value:** 1.052e-49\n\n3. **Calditerrivibrio nitroreducens DSM 19672, complete genome**\n   - **Species:** Calditerrivibrio nitroreducens\n   - **Chromosome:** Not specified\n   - **Location on Chromosome:** Positions 842,858 to 842,896\n   - **Identity Percentage:** 89.7%\n   - **Alignment Length:** 39 base pairs\n   - **E-value:** 0.062288

In [18]:
question_reverse = "Use ncbi database to answer the following question: What is the DNA sequence located at chr15:91950805-91950932"

process_question(question_reverse)

'**The DNA sequence is:**\n\n>NC_000015.10:91950805-91950932 Homo sapiens chromosome 15, GRCh38.p14 Primary Assembly\nATTCTGCCTTTAGTAATTTGATGACAGAGACTTCTTGGGAACCACAGCCAGGGAGCCACCCTTTACTCCA\nCCAACAGGTGGCTTATATCCAATCTGAGAAAGAAAGAAAAAAAAAAAAGTATTTCTCT'